In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json

# JSON 이란?
```
JavaScript Object Notation
키와 값으로 구성된 자바스크립트의 데이타 형태
파이썬의 JSON 레퍼런스 https://docs.python.org/ko/3/library/json.html

    
JSON 예시 - 키와 값의 구조
{
    "이름": "서태웅",
    "성별": "남",
    "학교": "북산 고등학교 1학년 10반 22번",
    "특기": "농구",
    "포지션": "스몰 포워드(SF)",
    "별명": "여우",
    "체격": {"키": "187 ㎝", "몸무게": " 75 ㎏", "혈액형": " AB형(Rh+)"},
    "라이벌": ["윤대협", "정우성"]
 }
 
 ```

## json 샘플 사이트 이용하기
- http://jsonplaceholder.typicode.com/
- url 요청 => json text 데이타 => 리스트 딕셔너리 => 데이타프레임

In [2]:
# (1) url 요청 => json_text
url = 'http://jsonplaceholder.typicode.com/todos'
res = requests.get(url)
print(res)
json_text = res.text
print(type(json_text))
# print(json_text)

<Response [200]>
<class 'str'>


In [3]:
# 2) json text 데이타 => 리스트 딕셔너리
# loads(json형태의텍스트)
# 리스트안에 딕셔너리 구조 
json_list = json.loads(json_text)
print(type(json_list))
# json_list

<class 'list'>


In [4]:
json_list[0], type(json_list[0])

({'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False},
 dict)

In [5]:
len(json_list)

200

In [6]:
# 리스트 딕셔너리 => 데이타프레임
df = pd.DataFrame(json_list)
df.sample(5)

,userId,id,title,completed
157,8,158,debitis vitae delectus et harum accusamus aut ...,True
190,10,191,temporibus atque distinctio omnis eius impedit...,True
160,9,161,ex hic consequuntur earum omnis alias ut occae...,True
171,9,172,et provident amet rerum consectetur et voluptatum,False
183,10,184,molestias modi perferendis perspiciatis,False


# Naver API 검색 서비스 

- 네이버 계정 보유 
- OPEN API 신청 : 네이버 개발자 센터 
```
https://developers.naver.com/main/

1) 네이버 계정으로 로그인 
https://developers.naver.com/main/

2) 검색 서비스 API 신청 
https://developers.naver.com/products/service-api/search/search.md
애플리케이션 등록 (API 이용신청)

3) 애플리케이션 등록 
- [Application]-[애플리케이션 등록] 

애플리케이션 이름
사용이름은? 검색 
환경 => 안드로이드 앱 패키지 이름

4) 클라이언트아이디와 클라이언트 시크릿 키 정보 확인 
2bPOmVkKFf8eygLmGiMk
DllokD9yvw
```

- 블로그 개발 가이드 
- https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EB%B8%94%EB%A1%9C%EA%B7%B8

In [8]:
# 네이버 검색 API예제는 블로그를 비롯 전문자료까지 호출방법이 동일하므로 blog검색만 대표로 예제를 올렸습니다.
# 네이버 검색 Open API 예제 - 블로그 검색
import os
import sys
import urllib.request

# 신청한 아이디와 시크릿으로 변경 
# client_id = "YOUR_CLIENT_ID"
# client_secret = "YOUR_CLIENT_SECRET"
client_id = "2bPOmVkKFf8eygLmGiMk"
client_secret = "DllokD9yvw"

# 요청변수 display 
# display	integer	N	10(기본값),100(최대)	검색 결과 출력 건수 지정
# 기본주소?요청변수1=값1&요청변수2=값2 
# https://openapi.naver.com/v1/search/blog?display=갯수&query=

# 검색할 단어를 입력하면 자동으로 주소 생성 
encText = urllib.parse.quote("오징어 게임")
# url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
url = "https://openapi.naver.com/v1/search/blog?display=50&query=" + encText # json 결과
# print(url)

# xml 용 주소 
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

# 페이지가 정상적으로 접근되어진다면 저장 그렇지 않으면 에러메세지와 코드 출력 
if(rescode==200):
    response_body = response.read()
#     print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

## json => 리스트 딕셔너리 

In [9]:
type(response_body)

bytes

In [10]:
# json.loads(response_body) => 딕셔너리 => 리스트 
blog_list = json.loads(response_body)['items']
print(type(blog_list))
len(blog_list)

<class 'list'>


50

In [11]:
blog_list[0]

{'title': '엔젤라 <b>오징어게임</b> 셀프 젤네일 스티커 후기',
 'link': 'https://blog.naver.com/a10000du?Redirect=Log&logNo=222556911073',
 'description': '타협해본 <b>오징어게임</b> 네일을 소개해보려합니다. 엔젤라 셀프 젤네일 스티커 [ 젤리블리 오징어 네일 ] 28pcs , 알콜패드, 미니파일, 설명서 동봉 2021년 할로윈 한정 에디션으로 나왔다고하는데 제가 직접 해보니... ',
 'bloggername': '만두로그',
 'bloggerlink': 'https://blog.naver.com/a10000du',
 'postdate': '20211102'}

In [12]:
# 리스트 딕셔너리 => 데이타프레임 
df_blog = pd.DataFrame(blog_list)
df_blog.head(3)

,title,link,description,bloggername,bloggerlink,postdate
0,엔젤라 <b>오징어게임</b> 셀프 젤네일 스티커 후기,https://blog.naver.com/a10000du?Redirect=Log&l...,타협해본 <b>오징어게임</b> 네일을 소개해보려합니다. 엔젤라 셀프 젤네일 스티커...,만두로그,https://blog.naver.com/a10000du,20211102
1,[40대삶의지혜]<b>오징어게임</b>으로 알아 본 사망보험금의 종류,https://blog.naver.com/sybangse?Redirect=Log&l...,하지만 오늘 기사를 보니 10년간 169억회의 유튜브 영상 조회수를 기록했던 왕좌의...,서울휘의 SideProject school,https://blog.naver.com/sybangse,20211126
2,"<b>오징어게임</b> 달고나만들기 세트 재료 식소다, 만드는방법",https://blog.naver.com/platonlove?Redirect=Log...,모양틀 위에 얹혀서 누르기판으로 살짝만 더 눌러주시면 모양뽑을 때 더 잘되요 ㅎ 모...,퐝언니 일상기록♥,https://blog.naver.com/platonlove,20211201


In [13]:
df_blog.columns

Index(['title', 'link', 'description', 'bloggername', 'bloggerlink',
       'postdate'],
      dtype='object')

In [14]:
# <b>~</b> 태그 없애기 
# 리스트 딕셔너리 => 데이타프레임 
# 1) 비어있는 데이타 프레임 생성 
df_blog2 = pd.DataFrame(columns=['title', 'link', 'description', 'postdate'])
df_blog2

,title,link,description,postdate


In [15]:
# 행이름 인덱스
idx = 1
for dict in blog_list:
    title = dict['title'].replace('<b>', '').replace('</b>', '')
    link = dict['link']
    description = dict['description'].replace('<b>', '').replace('</b>', '')
    postdate = dict['postdate']
    # 한행씩 추가 
    df_blog2.loc[idx] = [title, link, description, postdate]
    idx += 1

In [16]:
df_blog2.sample(3)

,title,link,description,postdate
21,오징어 게임 놀이 방법,https://blog.naver.com/bulsajo007?Redirect=Log...,즐겼던 오징어 게임인 것이다. &lt;게임 준비&gt; 먼저 서로 비슷한 상대가 되...,20211121
25,오징어게임 덕분에 달고나 만들기! 방법!,https://blog.naver.com/mayboom?Redirect=Log&lo...,#달고나 #오징어게임 #뽑기 #달고나게임 #달고나만들기 #달고나재료 #원픽 #꿀템 ...,20211127
6,할로윈데이 사탕 만들기 (feat. 오징어게임),https://blog.naver.com/wow_press?Redirect=Log&...,아주아주 핫한 오징어게임에서 영감을 얻어 귀염뽀짝하게 한 번 만들어봤습니다! &qu...,20211022


In [17]:
df_blog2.shape

(50, 4)

In [18]:
df_blog2.to_csv('output/blog_오징어게임.csv')

## 퀴즈  : 네이버 책 검색 API 


https://developers.naver.com/docs/serviceapi/search/book/book.md#%EC%B1%85

- 함수로 정의 
- 함수 실행시 검색어(책제목) => ?
- 검색어에 대한 검색 결과가 데이타 프레임으로 출력
- 검색 결과의 갯수는 임의 지정 

```
# 함수 호출 
book_api()

검색어(책제목) => ?

```

In [19]:
def book_api(book_name, max_num):
    
    encText = urllib.parse.quote(book_name)
    sep = round(int(max_num)/50)+1
    temp = []
    
    for a in range(0, sep):
        a = a * 50 + 1
        url = "https://openapi.naver.com/v1/search/book.json?start={}&display=50&query=".format(a) + encText # json 결과
    
    
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
    
        # 페이지가 정상적으로 접근되어진다면 저장 그렇지 않으면 에러메세지와 코드 출력 
        if(rescode==200):
            response_body = response.read()
#     print(response_body.decode('utf-8'))
        else:
            print("Error Code:" + rescode)
        
        # json.loads(response_body) => 딕셔너리 => 리스트 
        book_list = json.loads(response_body)['items']
        books = pd.DataFrame(book_list)
        temp.append(books)        
        
    df = pd.concat(temp, axis=0).reset_index(drop=True)
    
    return df
    

In [20]:
book_api("딥러닝", 100)

,title,link,image,author,price,discount,publisher,pubdate,isbn,description
0,혼자 공부하는 머신러닝 + <b>딥러닝</b> (1:1 과외하듯 배우는 인공지능 자습서),http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/175/...,박해선,26000,23400,한빛미디어,20201221,116224366X 9791162243664,"1:1 과외하듯 배우는 인공지능 자습서이 책은 수식과 이론으로 중무장한 머신러닝, ..."
1,밑바닥부터 시작하는 <b>딥러닝</b>. 3 (파이썬으로 직접 구현하며 배우는 <b...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/173/...,사이토 고키,38000,34200,한빛미디어,20201110,1162243597 9791162243596,코드 3줄이 <b>딥러닝</b> 프레임워크가 되는 마법\n이 책은 ‘밑바닥부터’ 직...
2,밑바닥부터 시작하는 <b>딥러닝</b> (파이썬으로 익히는 <b>딥러닝</b> 이론...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/114/...,사이토 고키,24000,21600,한빛미디어,20170103,8968484635 9788968484636,직접 구현하고 움직여보며 익히는 가장 쉬운 <b>딥러닝</b> 입문서!『밑바닥부터 ...
3,밑바닥부터 시작하는 <b>딥러닝</b> 2 (파이썬으로 직접 구현하며 배우는 순환 ...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/147/...,사이토 고키,29000,26100,한빛미디어,20190501,1162241748 9791162241745,직접 구현하면서 배우는 본격 <b>딥러닝</b> 입문서\n이번에는 순환 신경망과 자...
4,퀀트 투자를 위한 머신러닝·<b>딥러닝</b> 알고리듬 트레이딩 2/e(개정판) (...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/209/...,스테판 젠슨,50000,45000,에이콘출판,20210930,1161755691 9791161755694,머신러닝과 <b>딥러닝</b> 기술을 이용해 알고리듬 트레이딩의 아이디어에서 실행까...
...,...,...,...,...,...,...,...,...,...,...
145,<b>딥러닝</b>을 통한 정신의학 분야의 소규모 및 빅데이터 연구,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/164/...,Georgia Koppe,30000,,아진,20200723,8957616942 9788957616949,"기계학습, 특히 심층학습의 현대 알고리즘과 접근방식은 다른 학문에서의 탁월한 예측 ..."
146,"케라스로 구현하는 <b>딥러닝</b>과 강화학습 (신경망 기초부터 CNN, RNN,...",http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/126/...,안토니오 걸리|수짓 팔,30000,,에이콘출판,20171123,1161750746 9791161750743,케라스로 다양한 문제를 해결하면서 케라스와 <b>딥러닝</b> 기초를 쉽게 배울 수...
147,<b>딥러닝</b> : 코로나 -19 바이러스 중환자 조기 진단,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/164/...,Wenhua Liang|Jianhua Yao|Ailan Chen,20000,,아진,20200722,8957616934 9788957616932,코로나19 바이러스병(COVID-19) 환자가 갑자기 중증질환으로 악화한 것이 주요...
148,<b>딥러닝</b> 기반의 건화물선 시황예측 연구,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/159/...,윤희성,6000,6000,한국해양수산개발원,20191107,1189964376 9791189964375,▶ 이 책은 한국해양수산개발원의 <b>딥러닝</b> 기반의 건화물선 시황예측 연구를...
